In [1]:
# Import the necessary libraries
import syft as sy
sy.requires(">=0.8-beta")
import jax
import jax.numpy as jnp
import numpy as np

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/teo/OpenMined/PySyft
/home/teo/anaconda3/envs/PySyft/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


✅ The installed version of syft==0.8.1b1 matches the requirement >=0.8b0


In [2]:
# Launch the domain
node = sy.orchestra.launch(name="test-domain-1", reset=True)
domain_client = node.login(email="info@openmined.org", password="changethis")

SQLite Store Path:
!open file:///tmp/7bca415d13ed1ec841f0d0aede098dbb.sqlite



In [3]:
# Import some additional JAX and dataloader helpers
from jax.scipy.special import logsumexp
from jax.experimental import optimizers

import torch
from torchvision import datasets, transforms

import time
from helpers import plot_mnist_examples

/home/teo/anaconda3/envs/PySyft/lib/python3.10/site-packages/jax/experimental/optimizers.py:28: FutureWarning: jax.experimental.optimizers is deprecated, import jax.example_libraries.optimizers instead
  warnings.warn('jax.experimental.optimizers is deprecated, '


In [4]:
# Set the PyTorch Data Loader for the training & test set
batch_size = 100

train_dataset = datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

test_dataset = datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))


# plot_mnist_examples(train_loader)

In [12]:
np.asanyarray(test_dataset.data)

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [6]:
np.ones(np.asanyarray(train_dataset.data).shape)

array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       ...,

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1.

In [7]:
# Add a dummy data subject
group_subject = sy.DataSubject(name="Group", aliases=["group"])
train_subject = sy.DataSubject(name=f"TrainSubject", aliases=[f"group:train_subject"])
test_subject = sy.DataSubject(name=f"TestSubject", aliases=[f"group:test_subject"])
group_subject.add_member(train_subject)
group_subject.add_member(test_subject)
registry = domain_client.data_subject_registry
response = registry.add_data_subject(group_subject)
response

<class 'syft.service.response.SyftSuccess'>: 3 Data Subjects Registered

In [8]:
# Check the data subjects
data_subjects = domain_client.api.services.data_subject.get_all()
data_subjects

,type,id
0,<DataSubject: TestSubject>,555802cfd8274af89dc232e68b3fdcb1
1,<DataSubject: Group>,99b141af75f54fbfb01ed6f5a8dd6233
2,<DataSubject: TrainSubject>,aec0a51c755b40ff93803db9924d800c


In [17]:
# Add the needed data for the tutorial
# Optional add description, citation, url, contributors
dataset = sy.Dataset(name="MNIST Dataset")
dataset.set_description("MNIST Dataset Chapter 10")

train_asset = sy.Asset(name='train_data')
train_asset.set_description("train data for MNIST")
train_asset.set_obj(np.asanyarray(train_dataset.data))
train_asset.set_mock(np.ones(np.asanyarray(train_dataset.data).shape), mock_is_real=False)
train_asset.add_data_subject(train_subject)
train_asset.set_shape(np.asanyarray(test_dataset.data).shape)
dataset.add_asset(train_asset)

test_asset = sy.Asset(name='test_data')
test_asset.set_description("test data for MNIST")
test_asset.set_obj(np.asanyarray(test_dataset.data))
test_asset.set_mock(np.ones(np.asanyarray(test_dataset.data).shape), mock_is_real=False)
test_asset.add_data_subject(test_subject)
test_asset.set_shape(np.asanyarray(test_dataset.data).shape)
dataset.add_asset(test_asset)

In [18]:
# Upload the dataset
domain_client.upload_dataset(dataset)

  0%|          | 0/2 [00:00<?, ?it/s]

Uploading: train_data


 50%|█████     | 1/2 [00:01<00:01,  1.84s/it]

Uploading: test_data


100%|██████████| 2/2 [00:02<00:00,  1.10s/it]


<class 'syft.service.response.SyftSuccess'>: Dataset Added

# Review Projects